# Run Torch2ONNX from Arachne

Here, we explain how to use the Torch2ONNX tool from Arachne.

## Prepare a Model

First, we have to prepare a model to be used in this tutorial.
Here, we will use a pre-trained model of the ResNet-18 from `torchvision.models`.

In [1]:
import torch
import torchvision

resnet18 = torchvision.models.resnet18(pretrained=True)
torch.save(resnet18, f="/tmp/resnet18.pt")

## Run Torch2ONNX from Arachne

Now, let's convert the model into an ONNX model by Arachne.
To use the tool, we have to specify `+tools=torch2onnx` to `arachne.driver.cli`.
Available options can be seen by adding `--help`.

In [2]:
%%bash

python -m arachne.driver.cli +tools=torch2onnx --help

cli is powered by Hydra.

== Configuration groups ==
Compose your configuration from those groups (group=option)

tools: onnx_simplifier, onnx_tf, openvino2tf, openvino_mo, tflite_converter, tftrt, torch2onnx, torch2trt, tvm
tvm_target: dgx-1, dgx-s, jetson-nano, jetson-xavier-nx, rasp4b64


== Config ==
Override anything in the config (foo.bar=value)

model_file: null
model_dir: null
model_spec_file: null
output_path: ???
tools:
  torch2onnx:
    export_params: true
    verbose: false
    training: 0
    operator_export_type: null
    opset_version: 9
    do_constant_folding: false
    dynamic_axes: null
    keep_initializers_as_inputs: null
    custom_opsets: null


Powered by Hydra (https://hydra.cc)
Use --hydra-help to view Hydra specific help




Passing the Pytorch model and it's tensor specification, the tool will covert the model into an ONNX model by the following command.

In [4]:
yml = """
inputs:
  - dtype: float32
    name: input
    shape:
    - 1
    - 3
    - 224
    - 224
outputs:
  - dtype: float32
    name: output
    shape:
    - 1
    - 16
    - 224
    - 224
"""
open("/tmp/resnet18.yaml", "w").write(yml)

184

In [5]:
%%bash

python -m arachne.driver.cli +tools=torch2onnx model_file=/tmp/resnet18.pt model_spec_file=/tmp/resnet18.yaml output_path=/tmp/output.tar

## Run Torch2ONNX from Arachne Python Interface

The following code shows an example of using the tool from Arachne Python interface.

In [6]:
from arachne.data import ModelSpec, TensorSpec
from arachne.utils.model_utils import init_from_file, save_model
from arachne.tools.torch2onnx import Torch2ONNX, Torch2ONNXConfig

model_file_path = "/tmp/resnet18.pt"
input = init_from_file(model_file_path)
spec = ModelSpec(
    inputs=[TensorSpec(name="input0", shape=[1, 3, 224, 224], dtype="float32")],
    outputs=[TensorSpec(name="output0", shape=[1, 1000], dtype="float32")],
)
input.spec = spec

cfg = Torch2ONNXConfig()

output = Torch2ONNX.run(input, cfg)

save_model(model=output, output_path="/tmp/output.tar")